In [2]:
#%% import libraries 
import numpy as np
import os
from scipy import io
import re
import h5py
import math
import matplotlib.pyplot as plt
# from global_scripts import spiketools
import gc
# import torch
from tqdm import tqdm
import warnings
from typing import Tuple
import pickle
import math
import hdf5storage
import cv2
import sys

sys.path.append('/Users/niloughazavi/Documents/GitHub/CNN_data')
from WN_functions import *

In [3]:
# Experiment date
expDate = '20240229C'

# Temporal Resolution: upsampling/downsampling factor determines how finely we want to divide the time axis when processing the stimulus 
# UP_SAMP=1 --> no changes in temporal resolution and UP_SAMP=0 --> temporal resolution changes 
UP_SAMP = 0

# Spike Times-Gaussian Kernel Convolution 
# duration of each time bin (smaller time bin = higher temporal resolution for detecting rapid changes) 
bin_width=8
# sig=4 for bin_width=8 ms, sig=2 for bin_width=16 ms, sig=8 for bin_width=4 ms, sig=20 for bin_width=1 ms
sig=4 

# Light Levels 
lightLevel_text = ['scotopic','mesopic','photopic']
# multiplication factor for each light level 
lightLevel_mul = [0.5,50,6000]
# unit conversion of light level (R* excited state of molecule R) 
CONVERT_RSTAR = True
# stimulus normalization
NORM_STIM = 1
# RGC response normalization
NORM_RESP = True
FRAME_RESIZE = 8

# Stimulus 
# refresh rate is 60 Hz, spike bin rate is 120 Hz, the stimulus should be upsampled so each frame is 8 ms ( the same as bin time)
stim_upsample = 2 
# refresh rate for each frame ( t_frame should be the same as bin time after doubling the temporal resolution (stim_upsample)
t_frame=(1/60.3180)*1000/stim_upsample

In [14]:
# DOVEs MOVIE
# total number of movies (unique indices?)
total_num_movies= 161

# validation dataset
idx_movs_val=[0]

# Train dataset: Exclude the index of the validation dataset 
idx_movs_train= np.setdiff1d(np.arange(0,total_num_movies),idx_movs_val)

# number of validation movies
N_VAL = len(idx_movs_val)   # 2 movies for validation purpose

In [13]:
# directories to movies and raw data

path_raw = os.path.join('/Users/niloughazavi/Documents/Mike_Data/Gradients/RGC_Selective_Stimulation/Natural_Movies/analyses_parasol_midget_cells/data_mike_nat',expDate,'raw')
path_movs = os.path.join(path_raw,'movie_files')
path_db = os.path.join('/Users/niloughazavi/Documents/Mike_Data/Gradients/RGC_Selective_Stimulation/Natural_Movies/analyses_parasol_midget_cells/data_mike_nat',expDate,'datasets')
path_save = os.path.join('/Users/niloughazavi/Documents/Mike_Data/Gradients/RGC_Selective_Stimulation/Natural_Movies/analyses_parasol_midget_cells/data_mike_nat',expDate,'datasets')

for path in [path_raw, path_movs, path_db, path_save]:
    if not os.path.exists(path):
        os.makedirs(path)


# create files to save spike rate

if CONVERT_RSTAR==True:
    fname_save = os.path.join(path_save,(expDate+'_dataset_NATSTIM'+str(idx_movs_val[0])+'_CORR_'+'allLightLevels'+'_'+str(bin_width)+'ms'+'_Rstar.h5'))
else:
    fname_save = os.path.join(path_save,(expDate+'_dataset_NATSTIM'+str(idx_movs_val[0])+'_CORR_'+'allLightLevels'+'_'+str(bin_width)+'ms.h5'))


if not os.path.exists(path_save):
    os.mkdir(path_save)

In [ ]:
# Define the data for each sheet
off_parasol_data = {
    'Source_ID': [1, 125, 170, 181, 240, 268, 308, 316, 364, 395, 429, 442, 491, 500, 539, 559, 580, 598, 608, 666, 704, 720, 725, 766, 803, 818, 843, 853, 881, 886, 962, 995, 997, 1050, 1102, 1126, 1151, 1164, 1243, 1245, 1246, 1335, 1368, 1390, 1420, 1456, 1494, 1521, 1523, 1569, 1619, 1656, 1722, 1727, 1770],
    'Destination_ID': [1, 97, 161, 145, 193, 208, 234, 1487, 279, 1680, 339, 357, 404, 408, 435, 451, 467, 481, 496, 541, 571, 591, 595, 628, 663, 678, 705, 718, 752, 754, 801, 835, 831, 882, 921, 946, 971, 977, 1048, 1050, 1052, 1120, 1149, 1173, 1196, 1227, 1244, 1238, 1268, 1307, 1347, 1380, 1427, 1432, 183]
}

off_midget_data = {
    'Source_ID': [59, 101, 180, 188, 201, 237, 339, 346, 361, 406, 422, 477, 504, 534, 542, 548, 560, 562, 572, 576, 584, 779, 794, 808, 817, 821, 842, 857, 876, 931, 963, 975, 1016, 1085, 1131, 1174, 1178, 1193, 1215, 1277, 1455, 1489, 1543, 1558, 1598, 1623, 1629, 1636, 1672, 1702, 1717],
    'Destination_ID': [43, 68, 146, 153, 162, 192, 257, 263, 277, 316, 334, 389, 409, 429, 437, 443, 453, 454, 462, 466, 468, 638, 652, 668, 677, 681, 702, 1700, 746, 783, 802, 813, 846, 930, 951, 986, 989, 1007, 1042, 1075, 1226, 1240, 1286, 1301, 1302, 1349, 1356, 1367, 1390, 1404, 1423]
}

on_parasol_data = {
    'Source_ID': [19, 183, 433, 456, 464, 537, 547, 552, 615, 621, 656, 665, 684, 765, 785, 795, 845, 892, 926, 930, 1005, 1030, 1080, 1115, 1198, 1212, 1255, 1256, 1317, 1419, 1433, 1452, 1498, 1550, 1603, 1634, 1663, 1703, 1715, 1724, 1940],
    'Destination_ID': [103, 149, 342, 367, 377, 433, 282, 447, 502, 509, 531, 536, 556, 624, 644, 654, 706, 757, 780, 781, 841, 862, 907, 933, 1010, 1029, 1058, 1060, 1109, 1195, 1206, 1222, 1250, 1291, 1330, 1362, 1728, 1406, 1420, 1405, 139]
}

on_midget_data = {
    'Source_ID': [178, 418, 783, 826, 841, 847, 852, 896, 946, 1004, 1019, 1042, 1069, 1103, 1104, 1111, 1118, 1124, 1148, 1154, 1183, 1190, 1221, 1236, 1247, 1275, 1279, 1328, 1338, 1427],
    'Destination_ID': [144, 328, 640, 682, 701, 709, 717, 760, 792, 840, 849, 874, 901, 922, 923, 931, 938, 944, 970, 972, 998, 1005, 1035, 1043, 1053, 1074, 1082, 1119, 1124, 1201]
}


## DOVES Movie (Quality controlled cells)
ON Midget: array([ 682,  701,  709,  717,  792,  840,  849,  874,  901,  923,  931,
        938,  998, 1005, 1035, 1053, 1074])

OFF Midget: array([ 146,  153,  192,  277,  316,  389,  429,  443,  453,  454,  468,
        638,  681,  702,  783,  802,  846,  989, 1042, 1240, 1349, 1404,
       1423])

ON Parasol: array([ 509,  556,  624,  654,  706,  781,  862,  907, 1029, 1060, 1109,
       1195, 1206, 1222, 1291, 1362])

OFF Parasol : array([ 435,  451,  467,  496,  571,  628,  801,  831,  882,  971,  977,
       1050, 1149, 1307, 1347, 1427, 1432])

In [15]:

# cell IDs
photopic_cell_dove = {
'OffM': [43, 68, 146, 153, 162, 192, 257, 263, 277, 316, 334, 389, 409, 429, 437, 443, 453, 454, 462, 466, 468, 638, 652, 668, 677, 681, 702, 1700, 746, 783, 802, 813, 846, 930, 951, 986, 989, 1007, 1042, 1075, 1226, 1240, 1286, 1301, 1302, 1349, 1356, 1367, 1390, 1404, 1423],
'OffP': [1, 97, 161, 145, 193, 208, 234, 1487, 279, 1680, 339, 357, 404, 408, 435, 451, 467, 481, 496, 541, 571, 591, 595, 628, 663, 678, 705, 718, 752, 754, 801, 835, 831, 882, 921, 946, 971, 977, 1048, 1050, 1052, 1120, 1149, 1173, 1196, 1227, 1244, 1238, 1268, 1307, 1347, 1380, 1427, 1432, 183],
'OnM': [144, 328, 640, 682, 701, 709, 717, 760, 792, 840, 849, 874, 901, 922, 923, 931, 938, 944, 970, 972, 998, 1005, 1035, 1043, 1053, 1074, 1082, 1119, 1124, 1201],
'OnP': [103, 149, 342, 367, 377, 433, 282, 447, 502, 509, 531, 536, 556, 624, 644, 654, 706, 757, 780, 781, 841, 862, 907, 933, 1010, 1029, 1058, 1060, 1109, 1195, 1206, 1222, 1250, 1291, 1330, 1362, 1728, 1406, 1420, 1405, 139],
}



photopic_off_m = photopic_cell_dove['OffM']
photopic_off_p = photopic_cell_dove['OffP']
photopic_on_m = photopic_cell_dove['OnM']
photopic_on_p = photopic_cell_dove['OnP']


# only parasol cells 
uname_all = list()
ctr = 0
for i in photopic_on_p:
    ctr+=1
    uname_rgb = 'on_par_%03d'%ctr
    uname_all.append(uname_rgb)
    
ctr = 0
for i in photopic_off_p:
    ctr+=1
    uname_rgb = 'off_par_%03d'%ctr
    uname_all.append(uname_rgb)

ctr = 0
for i in photopic_on_m:
    ctr+=1
    uname_rgb = 'on_m_%03d'%ctr
    uname_all.append(uname_rgb)

ctr = 0
for i in photopic_off_m:
    ctr+=1
    uname_rgb = 'off_m_%03d'%ctr
    uname_all.append(uname_rgb)



totalNum_units = len(photopic_on_p) + len(photopic_off_p)+len(photopic_on_m)+len(photopic_off_m)


photopic_ids=photopic_on_p+photopic_off_p+photopic_on_m+photopic_off_m
photopic_ids=np.array(photopic_ids)

idx_allunits=photopic_ids


In [19]:
# how to update this 
spikerate_grand = np.empty((0,totalNum_units,9,10))      # [time,cells,stims,trials]   <-- nee to find a way to do this auto



lightLevel_idx = 2

for lightLevel_idx in [2]:     #[0=0.3; 1=3; 2=30]
    select_lightLevel = lightLevel_text[lightLevel_idx]

    fname = os.path.join(path_raw,expDate+'_Doves_'+select_lightLevel+'.pkl')
    with open(fname,'rb') as f:
        params = pickle.load(f)
    
    #---- Stim 
    
    spike_dict = params['spike_dict']
    cluster_id = params['cluster_id']
    stimulus_index = params['stimulusIndex'].astype(int)
    background_intensity = params['backgroundIntensity'].astype(float)
    frame_times = params['frame_times'] # This is the time of each frame in milliseconds.
    pre_pts = params['pre_pts']
    stim_pts = params['stim_pts']
    tail_pts = params['tail_pts']
    wait_time = params['wait_time']
    bin_rate = params['bin_rate'] # This is the bin rate for the spike times in Hz.


    unique_movs = np.unique(stimulus_index)
    N_TRIALS = int(len(stimulus_index)/len(unique_movs))
    frames_mat = np.zeros((362,int(600/FRAME_RESIZE),int(800/FRAME_RESIZE),len(unique_movs)));frames_mat[:] = np.nan        # [time,y/8,x/8,img,trial]
    spikes_mat = np.zeros((len(unique_movs),N_TRIALS),dtype='object')
    ctr_trial = -1*np.ones(N_TRIALS,dtype='int')
    
    # Loop over movies
    m=5
    for m in tqdm(range(len(unique_movs))):
        movie_idx = unique_movs[m]
        movie_loc = np.where(movie_idx==unique_movs)[0][0]
        


        single_movie_path = os.path.join(path_movs, 'doves_img_' + str(movie_idx) + '.npy')
        frames = np.load(single_movie_path)

        # how to add the mean
        bg_mean = movie_dic['image_mean']
        frames = frames[:,::FRAME_RESIZE,::FRAME_RESIZE,0]
        frames = (frames+frames.max())/(frames.max()-frames.min())
        
        frames_new = (frames - bg_mean) / bg_mean       # convert to  contrast
        
        frames_mat[:,:,:,movie_loc] = frames_new

        frame_rate = 59.94154881781792


    if CONVERT_RSTAR == True:
        meanIntensity = lightLevel_mul[lightLevel_idx]
        frames_rstar = applyLightIntensities(meanIntensity,frames_mat,t_frame)
        
    stim_frames = np.repeat(frames_rstar,stim_upsample,axis=0).astype('float32')
    stim_frames = np.repeat(stim_frames[:,:,:,:,None],N_TRIALS,axis=-1)

    # Extract spikes
    epoch_count = 0
    for epoch_count in tqdm(range(len(stimulus_index))):
        movie_idx = int(stimulus_index[epoch_count])
        movie_loc = np.where(movie_idx==unique_movs)[0][0]
        ctr_trial[movie_loc] += 1

        epoch_frame_times = np.array(frame_times[epoch_count])  # Get the frame times for the epoch.
        epoch_frame_times -= epoch_frame_times[0]               # Subtract the first frame.
        stim_start = pre_pts[epoch_count]                       # The stimulus starts at the end of pre_pts.
        the_wait_is_over = stim_start + wait_time[epoch_count]  # The first movie frame is then presented for the duration of the wait time.
        stim_end = stim_start + stim_pts[epoch_count]           # The stimulus then plays for the duration of the stim_pts-wait_time.
    

    





  0%|                                                   | 0/161 [00:00<?, ?it/s]


NameError: name 'movie_dic' is not defined

In [ ]:
# how to update this 
spikerate_grand = np.empty((0,totalNum_units,9,10))      # [time,cells,stims,trials]   <-- nee to find a way to do this auto



lightLevel_idx = 2

for lightLevel_idx in [2]:     #[0=0.3; 1=3; 2=30]
    select_lightLevel = lightLevel_text[lightLevel_idx]

    fname = os.path.join(path_raw,expDate+'_Doves_'+select_lightLevel+'.pkl')
    with open(fname,'rb') as f:
        params = pickle.load(f)
    
    #---- Stim 
    
    spike_dict = params['spike_dict']
    cluster_id = params['cluster_id']
    stimulus_index = params['stimulusIndex'].astype(int)
    background_intensity = params['backgroundIntensity'].astype(float)
    frame_times = params['frame_times'] # This is the time of each frame in milliseconds.
    pre_pts = params['pre_pts']
    stim_pts = params['stim_pts']
    tail_pts = params['tail_pts']
    wait_time = params['wait_time']
    bin_rate = params['bin_rate'] # This is the bin rate for the spike times in Hz.


    unique_movs = np.unique(stimulus_index)
    N_TRIALS = int(len(stimulus_index)/len(unique_movs))
    frames_mat = np.zeros((362,int(600/FRAME_RESIZE),int(800/FRAME_RESIZE),len(unique_movs)));frames_mat[:] = np.nan        # [time,y/8,x/8,img,trial]
    spikes_mat = np.zeros((len(unique_movs),N_TRIALS),dtype='object')
    ctr_trial = -1*np.ones(N_TRIALS,dtype='int')
    
    # Loop over movies
    m=5
    for m in tqdm(range(len(unique_movs))):
        movie_idx = unique_movs[m]
        movie_loc = np.where(movie_idx==unique_movs)[0][0]
        


        single_movie_path = os.path.join(path_movs, 'doves_idx_' + str(movie_idx) + '.npy')
        frames = np.load(single_movie_path)
    
        bg_mean = movie_dic['image_mean']
        # frames = cv2.resize(frames, dsize=(int(frames.shape[1]/FRAME_RESIZE), int(frames.shape[0]/FRAME_RESIZE)), interpolation=cv2.INTER_LINEAR)
        frames = frames[:,::FRAME_RESIZE,::FRAME_RESIZE,0]
        frames = (frames+frames.max())/(frames.max()-frames.min())
        
        frames_new = (frames - bg_mean) / bg_mean       # convert to  contrast
        
        frames_mat[:,:,:,movie_loc] = frames_new

        frame_rate =  59.94154881781792


    if CONVERT_RSTAR == True:
        meanIntensity = lightLevel_mul[lightLevel_idx]
        frames_rstar = applyLightIntensities(meanIntensity,frames_mat,t_frame)
        
    stim_frames = np.repeat(frames_rstar,stim_upsample,axis=0).astype('float32')
    stim_frames = np.repeat(stim_frames[:,:,:,:,None],N_TRIALS,axis=-1)

    # Extract spikes
    epoch_count = 0
    for epoch_count in tqdm(range(len(stimulus_index))):
        movie_idx = int(stimulus_index[epoch_count])
        movie_loc = np.where(movie_idx==unique_movs)[0][0]
        ctr_trial[movie_loc] += 1

        epoch_frame_times = np.array(frame_times[epoch_count])  # Get the frame times for the epoch.
        epoch_frame_times -= epoch_frame_times[0]               # Subtract the first frame.
        stim_start = pre_pts[epoch_count]                       # The stimulus starts at the end of pre_pts.
        the_wait_is_over = stim_start + wait_time[epoch_count]  # The first movie frame is then presented for the duration of the wait time.
        stim_end = stim_start + stim_pts[epoch_count]           # The stimulus then plays for the duration of the stim_pts-wait_time.
    
        # Get the relevant spikes
        spikes_epoch = spike_dict[:,epoch_count]
        spikes_stim = []
        i=0
        for i in range(len(spikes_epoch)):
             rgb = spikes_epoch[i]
             rgb = rgb[rgb>stim_start]
             rgb = rgb[rgb<stim_end]
             rgb = rgb - stim_start
             rgb = rgb[rgb>0]
             spikes_stim.append(rgb)
        spikes_mat[movie_loc,ctr_trial[movie_loc]] = spikes_stim
    
    

    # ---- spikes
    assert len(np.unique(stim_pts))==1, 'stim_pts different for different epochs/movies'

    stimLength = stim_frames.shape[0]
    
    flips = np.arange(0,(stimLength+1)*t_frame,t_frame)
    
    numStims = len(unique_movs)
    spikeRate_cells = np.empty((stimLength,totalNum_units,numStims,N_TRIALS))
    spikeCounts_cells = np.empty((stimLength,totalNum_units,numStims,N_TRIALS))
    spikeTimes_cells = np.empty((totalNum_units,numStims,N_TRIALS),dtype='object')
    
    
    ctr_units = -1
    U = 0
    # had to be modified for one light level
    for U in range(0, len(idx_allunits)):
        ctr_units += 1
        cluster_unit = idx_allunits[U]
        idx_unit_array = np.where(cluster_id == cluster_unit)[0]
        # if len(idx_unit_array) == 0:
        #     continue  # Skip if cluster_unit is not found in cluster_id
        idx_unit = idx_unit_array[0]  # Convert array to integer
        
        tr = 0
        s = 0
        for s in range(numStims):
            for tr in range(N_TRIALS):
                startTime = 0  # Start time for stimulus
                endTime = startTime + flips[-1]  # End time for stimulus
                
                spikes = spikes_mat[s, tr][idx_unit]
                if len(spikes) == 0:
                    spikes = 0
                    spikeCounts = np.zeros(stimLength)
                else:
                    spikeCounts = np.histogram(spikes, flips)[0]  # Bin the spikes
                spikeRate = MEA_spikerates_binned(spikeCounts, sig)  # Calculate spike rates
                
                spikeRate_cells[:, ctr_units, s, tr] = spikeRate
                spikeCounts_cells[:, ctr_units, s, tr] = spikeCounts
                spikeTimes_cells[ctr_units, s, tr] = spikes
    


    # Normalize responses if required
    if NORM_RESP:
        rgb = np.squeeze(spikeRate_cells)
        rgb[rgb == 0] = np.nan
        # resp_median = np.nanmedian(rgb, axis=0)
        # nilou added
        if np.all(np.isnan(rgb)):
            resp_median = np.zeros_like(rgb[0])
        else:
            resp_median = np.nanmedian(rgb, axis=0)
        resp_norm = rgb / resp_median[None, :]
        resp_norm[np.isnan(resp_norm)] = 0
        resp_orig = spikeRate_cells
    else:
        resp_norm = spikeRate_cells
        resp_orig = spikeRate_cells

    # Prepare training and validation datasets
    idx_frames_start = 0  # Discard first 121 frames
    stim_frames_train = (stim_frames[idx_frames_start:, :, :, idx_movs_train, :], flips[idx_frames_start:])
    spikeRate_train = (resp_norm[idx_frames_start:, :, idx_movs_train, :], spikeCounts_cells[idx_frames_start:, :, idx_movs_train, :], spikeTimes_cells[:, idx_movs_train, :])
    spikeRate_orig_train = spikeRate_cells[idx_frames_start:, :, idx_movs_train, :]
    spikeRate_median_train = resp_median[:, idx_movs_train, :]
    unique_movs_train = unique_movs[idx_movs_train]

    stim_frames_val = (stim_frames[idx_frames_start:, :, :, idx_movs_val, :], flips[idx_frames_start:])
    spikeRate_val = (resp_norm[idx_frames_start:, :, idx_movs_val, :], spikeCounts_cells[idx_frames_start:, :, idx_movs_val, :], spikeTimes_cells[:, idx_movs_val, :])
    spikeRate_orig_val = spikeRate_cells[idx_frames_start:, :, idx_movs_val, :]
    spikeRate_median_val = resp_median[:, idx_movs_val, :]
    unique_movs_val = unique_movs[idx_movs_val]
    
    # Update grand spikerate
    spikerate_grand = np.concatenate((spikerate_grand, spikeRate_cells), axis=0)

    # Save dataset
    
        # Create directories if necessary before saving
    os.makedirs(os.path.dirname(fname_save), exist_ok=True)
    
    with h5py.File(fname_save, 'a') as f:
        try:
            # Save experimental date and unit information
            f.create_dataset('expDate', data=np.array(expDate, dtype='bytes'))
            f.create_dataset('units', data=np.array(uname_all, dtype='bytes'))
        except:
            pass  # If datasets already exist, skip creating them
    
        # Save training data
        grp = f.create_group('/' + select_lightLevel + '/train')
        create_dataset(grp, stim_frames_train, spikeRate_train, spikeRate_orig_train, spikeRate_median_train, unique_movs_train, t_frame, N_TRIALS)
    
        # Save validation data
        grp = f.create_group('/' + select_lightLevel + '/val')
        create_dataset(grp, stim_frames_val, spikeRate_val, spikeRate_orig_val, spikeRate_median_val, unique_movs_val, t_frame, N_TRIALS)
    
    # Flatten and reshape spike rate data for saving
    spikerate_grand_flattened = np.moveaxis(spikerate_grand, 0, -1)
    spikerate_grand_flattened = spikerate_grand_flattened.reshape(spikerate_grand_flattened.shape[0], -1)
    resp_median_grand = np.nanmedian(spikerate_grand_flattened, axis=-1)
    
    # Save grand spike rate and median response data
    with h5py.File(fname_save, 'a') as f:
        if 'spikerate_grand' in f:
            del f['spikerate_grand']  # Delete existing dataset if it exists
        if 'resp_median_grand' in f:
            del f['resp_median_grand']  # Delete existing dataset if it exists
        f.create_dataset('spikerate_grand', data=spikerate_grand_flattened, compression='gzip')
        f.create_dataset('resp_median_grand', data=resp_median_grand, compression='gzip')